In [ ]:
using Pkg
if isfile("../Project.toml") && isfile("../Manifest.toml")
    Pkg.activate("..");
    ENV["PYTHON"] = "python3";
end

function printer(x::Any, verbose="")
    if verbose != ""
        println(verbose)
    end
    show(stdout, "text/plain", x)
    println("")
end

using Plots, Random, Distributions, LinearAlgebra, JuliaProbo
gr();

In [ ]:
function ch06_kf3()
    dt = 0.1
    # environment
    xlim = [-5.0, 5.0]
    ylim = [-5.0, 5.0]
    # id of landmark must start from 0 with 1 step
    landmarks = [Landmark([-4.0, 2.0], 0), Landmark([2.0, -3.0], 1), Landmark([3.0, 3.0], 2)]
    envmap = Map()
    push!(envmap, landmarks)
    world = World(xlim, ylim)
    push!(world, envmap)
    # robot side
    initial_pose = [0.0, 0.0, 0.0]
    estimator1 = KalmanFilter(envmap, initial_pose)
    agent1 = EstimatorAgent(0.2, 10.0*pi/180, dt, estimator1)
    robot1 = RealRobot(initial_pose, agent1, RealCamera(landmarks); color="red")
    push!(world, robot1)
    
    estimator2 = KalmanFilter(envmap, initial_pose)
    agent2 = EstimatorAgent(0.1, 0.0, dt, estimator2)
    robot2 = RealRobot(initial_pose, agent2, RealCamera(landmarks); color="red")
    push!(world, robot2)
    
    estimator3 = KalmanFilter(envmap, initial_pose)
    agent3 = EstimatorAgent(0.1, -3.0/180*pi, dt, estimator3)
    robot3 = RealRobot(initial_pose, agent3, RealCamera(landmarks); color="red")    
    push!(world, robot3)
    
    anim = @animate for i in 1:300
        t = dt * i
        annota = "t = $(round(t, sigdigits=3))[s]"
        p = draw(world, annota)
        # robot1
        z1 = observations(robot1.sensor_, robot1.pose_; noise=true, bias=true)
        v1, ω1 = decision(agent1, z1, envmap)
        state_transition(robot1, v1, ω1, dt; move_noise=true, vel_bias_noise=true)
        # robot2
        z2 = observations(robot2.sensor_, robot2.pose_; noise=true, bias=true)
        v2, ω2 = decision(agent2, z2, envmap)
        state_transition(robot2, v2, ω2, dt; move_noise=true, vel_bias_noise=true)
        # robot3
        z3 = observations(robot3.sensor_, robot3.pose_; noise=true, bias=true)
        v3, ω3 = decision(agent3, z3, envmap)
        state_transition(robot3, v3, ω3, dt; move_noise=true, vel_bias_noise=true)
    end
    gif(anim, "images/ch06_kf3.gif", fps=10)
end
ch06_kf3()